# Spectral plotter

In [1]:
#Input file directories
input_files = [#Example files
    'data/28Tau/28Tau-Ha-600-1.fit',
    'data/28Tau/28Tau-Ha-600-2.fit',
    'data/28Tau/28Tau-Ha-600-3.fit'
]
#Directories of the dark images
dark_files = [#Example files
    'data/28Tau/dark/DARK600-001.fit',
    'data/28Tau/dark/DARK600-002.fit',
    'data/28Tau/dark/DARK600-003.fit',
]
output_location = 'data/28Tau/results/28Tau-Ha.fit'

In [3]:
# Prerequisites
from astropy.io import fits 
from astropy.table import Table 
import numpy as np 
import matplotlib.pyplot as plt 
import math
%matplotlib inline 

# Build lists of darks and their corresponding data
dark = list()
data_dark = list()
for i in range(len(dark_files)):
    dark.append( fits.open(dark_files[i]) )
    data_dark.append( dark[i][0].data )

#  Get header information of any dark
# (darks must be _identical_ in shape)
header = dark[0][0].header

# Get the master dark array from median of all darks
mdark_arr = np.median(data_dark, axis=0)

# Initialise lists of exposures and their corresponding data
inputs = list()
data_inputs = list()

# For each individual exposure, ...
for i in range(len(input_files)):
    # ... append the exposure to the list, ...
    inputs.append( fits.open(input_files[i]) )
    # ... and substract the master dark element-wise!
    data_inputs.append( inputs[i][0].data - mdark_arr )

# Get the master image from median of all exposures
output = np.median(data_inputs, axis=0)

# Save the master image 2D array as a fits file, ...
hdu=fits.PrimaryHDU(output)
hdulist=fits.HDUList([hdu])
# ... and write it to the desired location
hdulist.writeto(output_location, overwrite=True)

In [4]:
#Prerequisites for editing
input_file = output_location
# Range of y in which all of the signal is contained
rangelow = 560
rangeup = 610
# Upper bound from which pixels will be removed
cosm_filt = 110000

#lamp wavelengths - requires manual search for the wavelengths of the emission lines
line_lambda = [
    6598.95,
    6532.88,
    6506.30
]

lamp_file = 'data/28Tau/lamp/gammaCasHa-lamp.fit'
otpt_file = 'data/28Tau/results/28Tau.png'
max_otpt = 300
spectrum_y = 580

In [5]:
# Loading input file
datahdu = fits.open(input_file) 
data, hdr = datahdu[0].data, datahdu[0].header

# Initialising axes
y = range(hdr['NAXIS1'])

# Summing all 'good' pixels in a column
# (ones that are not cosmics or burnt out)
xdata = np.sum(data[rangelow:rangeup,:])
            
# Loading lamp file
lamp = fits.open(lamp_file)
lamp_data, lamp_hdr = lamp[0].data, lamp[0].header
lamp_1d = list()

# Get a horizontal line from the lamp file
for i in range(lamp_hdr['NAXIS1']):
    lamp_1d.append(lamp_data[spectrum_y,i])

# Get lamp peaks
lamp_len = len(lamp_1d)
peaks = list()
for i in range(lamp_len):
    if i!=0 and i!=lamp_len-1:
        # Local maximum check
        cond1 = float(lamp_1d[i])-float(lamp_1d[i-1]) < 0
        cond2 = float(lamp_1d[i+1]) - float(lamp_1d[i]) < 0
        # Check if local maximum is not noise (edit if there are more lines to fit)
        cond3 = lamp_1d[i] > 3000
        if (cond1 != cond2) and cond3:
            peaks.append(i)
            
# Calibrate lamp by fitting a line from maxima and wavelengths
poly = np.polyfit(peaks,line_lambda,1)
a, b = poly[0], poly[1]
print("Lamp calibration - fit parameters:")
print("a = {0:.4f}".format(a))
print("b = {0:.4f}".format(b))
print(f"Peaks: {peaks}")

img_len = len(xdata)
avg_list = list()
# Smooth calibration step
st = 3
# Get the size of the array with the given step
size = math.floor(img_len/st)
rng = np.arange(size*st,step = st)
for i in rng:
    if i+st-1 < img_len:
        # Get the median of [st] number of consecutive pixels
        avg_list.append(np.median(xdata[i:i+st-1]))
        
# Gets the range of wavelengths and cuts the first and last 5 data points
x = np.linspace(a*(st*size-5)+b,a*5+b,num = size-10)
# Reverse the dataset
data = avg_list[5:size-5]
data.reverse()

# Plot graph
plt.figure(figsize=(16,9))
plt.rcParams.update({'font.size':22})
plt.plot(x,data)
plt.xlabel(r"Wavelength ($\AA$)")
plt.ylabel("Relative Flux (ADU)")
plt.xlim(a*(st*size-5)+b,a*5+b)
plt.savefig(otpt_file)

Lamp calibration - fit parameters:
a = -0.1117
b = 6671.8863
Peaks: [652, 1247, 1480]


TypeError: object of type 'numpy.float64' has no len()